# Neural Style Transfer with PyTorch and VGG19

Welcome to this comprehensive tutorial on Neural Style Transfer! This notebook demonstrates how to combine the content of one image with the artistic style of another using deep learning.

## 📚 What You'll Learn

- Understanding the Neural Style Transfer algorithm
- How VGG19 extracts content and style features
- Implementing loss functions for content and style
- Optimizing images through gradient descent
- Experimenting with different parameters

## 🎯 Prerequisites

- Basic understanding of neural networks
- Familiarity with PyTorch
- Knowledge of image processing concepts

## 1. Setup and Imports

Let's start by importing all necessary libraries and our custom modules:

In [ ]:
# Standard library imports
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Third-party imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm

# Custom module imports
from neural_style_transfer import NeuralStyleTransfer, VGGFeatureExtractor
from utils import (
    load_and_preprocess_image, 
    tensor_to_pil, 
    save_image, 
    visualize_comparison,
    create_image_grid
)
from config import Config, PRESETS

# Set up matplotlib for better display
plt.style.use('default')
%matplotlib inline

print("✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 2. Understanding Neural Style Transfer

Neural Style Transfer was introduced by Gatys et al. in 2015. The key insight is that:

1. **Content Representation**: Higher layers of CNNs capture content information
2. **Style Representation**: Correlations between feature maps (Gram matrices) capture style
3. **Optimization**: We can optimize a generated image to match both representations

### The Algorithm Overview:
1. Extract content features from a content image using VGG19
2. Extract style features (Gram matrices) from a style image using VGG19
3. Start with a random image (or copy of content image)
4. Iteratively optimize the image to minimize both content and style loss

In [ ]:
# Let's visualize the VGG19 architecture and our feature extraction layers
def visualize_vgg_architecture():
    """Display the VGG19 architecture and highlight our feature extraction layers."""
    
    fig, ax = plt.subplots(1, 1, figsize=(14, 8))
    
    # VGG19 layer information
    layers = [
        ('Input', 'lightblue', 'Input Image\n224x224x3'),
        ('conv1_1', 'lightgreen', 'Conv 3x3, 64\nReLU\n224x224x64'),
        ('conv1_2', 'lightgray', 'Conv 3x3, 64\nReLU, MaxPool\n112x112x64'),
        ('conv2_1', 'lightgreen', 'Conv 3x3, 128\nReLU\n112x112x128'),
        ('conv2_2', 'lightgray', 'Conv 3x3, 128\nReLU, MaxPool\n56x56x128'),
        ('conv3_1', 'lightgreen', 'Conv 3x3, 256\nReLU\n56x56x256'),
        ('conv3_2-4', 'lightgray', 'Conv 3x3, 256\nReLU (x3), MaxPool\n28x28x256'),
        ('conv4_1', 'lightgreen', 'Conv 3x3, 512\nReLU\n28x28x512'),
        ('conv4_2', 'gold', 'Conv 3x3, 512\nReLU (CONTENT)\n28x28x512'),
        ('conv4_3-4', 'lightgray', 'Conv 3x3, 512\nReLU (x2), MaxPool\n14x14x512'),
        ('conv5_1', 'lightgreen', 'Conv 3x3, 512\nReLU\n14x14x512'),
        ('conv5_2-4', 'lightgray', 'Conv 3x3, 512\nReLU (x3), MaxPool\n7x7x512')
    ]
    
    y_positions = np.arange(len(layers))
    
    for i, (name, color, description) in enumerate(layers):
        # Draw rectangle for layer
        rect = plt.Rectangle((0, i-0.4), 2, 0.8, facecolor=color, edgecolor='black', linewidth=1)
        ax.add_patch(rect)
        
        # Add layer name
        ax.text(1, i, name, ha='center', va='center', fontweight='bold', fontsize=10)
        
        # Add description
        ax.text(3, i, description, ha='left', va='center', fontsize=9)
    
    # Add legend
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, facecolor='lightgreen', label='Style Layers'),
        plt.Rectangle((0, 0), 1, 1, facecolor='gold', label='Content Layer'),
        plt.Rectangle((0, 0), 1, 1, facecolor='lightgray', label='Other Layers')
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    ax.set_xlim(-0.5, 8)
    ax.set_ylim(-0.5, len(layers) - 0.5)
    ax.set_title('VGG19 Architecture for Neural Style Transfer', fontsize=16, fontweight='bold')
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("📊 VGG19 Feature Extraction Strategy:")
    print("• Style Layers (Green): conv1_1, conv2_1, conv3_1, conv4_1, conv5_1")
    print("• Content Layer (Gold): conv4_2")
    print("• Style is captured by Gram matrices of feature correlations")
    print("• Content is captured by direct feature comparison")

visualize_vgg_architecture()

## 3. Initialize the Model and Device

Let's set up our Neural Style Transfer model and check our computational resources:

In [ ]:
# Initialize device and model
device = Config.get_device(prefer_gpu=True)
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Create directories for our outputs
Config.create_directories()

# Initialize the Neural Style Transfer model
nst = NeuralStyleTransfer(device=device)
print("\n✅ Neural Style Transfer model initialized!")

# Display current configuration
print("\n🔧 Current Configuration:")
for key, value in Config.get_default_params().items():
    print(f"  {key}: {value}")

## 4. Load and Visualize Example Images

Let's load our example content and style images to see what we're working with:

In [ ]:
# Check if example images exist, if not we'll create simple ones
content_path = "examples/content_mountain.svg"
style_path = "examples/style_van_gogh.svg"

def create_sample_images():
    """Create sample images if examples don't exist."""
    
    # Create a simple content image
    content_img = Image.new('RGB', (400, 300), color='skyblue')
    # Add some simple shapes
    from PIL import ImageDraw
    draw = ImageDraw.Draw(content_img)
    
    # Mountains
    draw.polygon([(0, 200), (100, 100), (200, 150), (300, 80), (400, 120), (400, 300), (0, 300)], fill='gray')
    # Sun
    draw.ellipse([320, 40, 360, 80], fill='yellow')
    # Lake
    draw.ellipse([100, 220, 300, 250], fill='blue')
    
    content_img.save('temp_content.jpg')
    
    # Create a simple style image with patterns
    style_img = Image.new('RGB', (400, 300), color='purple')
    draw = ImageDraw.Draw(style_img)
    
    # Create swirl-like patterns
    for i in range(0, 400, 20):
        for j in range(0, 300, 20):
            color = (255 - i//2, 100 + j//3, 150 + i//3)
            draw.ellipse([i, j, i+15, j+15], fill=color)
    
    style_img.save('temp_style.jpg')
    
    return 'temp_content.jpg', 'temp_style.jpg'

# Try to load example images, create simple ones if they don't exist
try:
    if os.path.exists(content_path) and os.path.exists(style_path):
        # Convert SVG to PIL Image for display (simplified)
        print("📁 Using example SVG images")
        content_display_path = content_path
        style_display_path = style_path
    else:
        print("📁 Creating sample images for demonstration")
        content_display_path, style_display_path = create_sample_images()
        
except Exception as e:
    print(f"Creating fallback images: {e}")
    content_display_path, style_display_path = create_sample_images()

# Load images for processing
print("\n🖼️ Loading and preprocessing images...")
image_size = 512

if content_display_path.endswith('.svg'):
    # For SVG files, we'll need to render them first
    print("Note: SVG images detected. In a real scenario, you'd convert these to raster images first.")
    print("For this demo, we'll create sample raster images.")
    content_display_path, style_display_path = create_sample_images()

content_tensor = load_and_preprocess_image(content_display_path, target_size=image_size, device=device)
style_tensor = load_and_preprocess_image(style_display_path, target_size=image_size, device=device)

# Display the input images
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(tensor_to_pil(content_tensor))
axes[0].set_title('Content Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(tensor_to_pil(style_tensor))
axes[1].set_title('Style Image', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"✅ Images loaded successfully!")
print(f"Content image shape: {content_tensor.shape}")
print(f"Style image shape: {style_tensor.shape}")

## 5. Exploring Feature Extraction

Let's examine how VGG19 extracts features and understand what the network "sees" at different layers:

In [ ]:
# Extract features from both images
print("🔍 Extracting features from VGG19...")

with torch.no_grad():
    # Normalize images for VGG
    content_features = nst.feature_extractor(nst.normalize_vgg(content_tensor))
    style_features = nst.feature_extractor(nst.normalize_vgg(style_tensor))

print("\n📊 Feature Map Dimensions:")
for layer_name in nst.feature_extractor.content_layers + nst.feature_extractor.style_layers:
    if layer_name in content_features:
        shape = content_features[layer_name].shape
        print(f"  {layer_name}: {shape} ({shape[1]} channels, {shape[2]}x{shape[3]} spatial)")

def visualize_feature_maps(features, layer_name, title, max_channels=16):
    """Visualize feature maps from a specific layer."""
    if layer_name not in features:
        print(f"Layer {layer_name} not found")
        return
    
    feature_map = features[layer_name].squeeze(0)  # Remove batch dimension
    num_channels = min(feature_map.shape[0], max_channels)
    
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    fig.suptitle(f'{title} - {layer_name} Feature Maps', fontsize=16, fontweight='bold')
    
    for i in range(16):
        row, col = i // 4, i % 4
        if i < num_channels:
            # Normalize feature map for visualization
            fm = feature_map[i].cpu().numpy()
            fm = (fm - fm.min()) / (fm.max() - fm.min() + 1e-8)
            
            axes[row, col].imshow(fm, cmap='viridis')
            axes[row, col].set_title(f'Channel {i+1}', fontsize=10)
        else:
            axes[row, col].axis('off')
        axes[row, col].set_xticks([])
        axes[row, col].set_yticks([])
    
    plt.tight_layout()
    plt.show()

# Visualize feature maps from different layers
print("\n🎨 Content Features (conv4_2):")
visualize_feature_maps(content_features, 'conv4_2', 'Content Image')

print("\n🎭 Style Features (conv1_1):")
visualize_feature_maps(style_features, 'conv1_1', 'Style Image')

## 6. Understanding Gram Matrices for Style

The Gram matrix captures style by measuring correlations between different feature channels:

In [ ]:
from neural_style_transfer import GramMatrix

def visualize_gram_matrices(features, layers, title_prefix):
    """Visualize Gram matrices for style representation."""
    gram_computer = GramMatrix()
    
    fig, axes = plt.subplots(1, len(layers), figsize=(15, 3))
    if len(layers) == 1:
        axes = [axes]
    
    for i, layer in enumerate(layers):
        if layer in features:
            # Compute Gram matrix
            gram = gram_computer(features[layer])
            gram_np = gram.squeeze(0).cpu().numpy()
            
            # Visualize Gram matrix
            im = axes[i].imshow(gram_np, cmap='coolwarm', aspect='auto')
            axes[i].set_title(f'{title_prefix}\n{layer}', fontsize=12)
            axes[i].set_xlabel('Feature Channel')
            axes[i].set_ylabel('Feature Channel')
            
            # Add colorbar
            plt.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)
    
    plt.suptitle('Gram Matrices (Style Representation)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize Gram matrices for style layers
style_layers_subset = ['conv1_1', 'conv2_1', 'conv3_1']  # First three for visualization
print("🎭 Style Representation via Gram Matrices:")
visualize_gram_matrices(style_features, style_layers_subset, 'Style Image')

# Show difference in Gram matrices
print("\n📈 Understanding Gram Matrices:")
print("• Each cell (i,j) represents correlation between feature channels i and j")
print("• Diagonal elements show feature channel variances")
print("• Off-diagonal elements show feature channel covariances")
print("• Different artistic styles produce different correlation patterns")
print("• The Gram matrix is texture/style invariant to spatial location")

## 7. Interactive Style Transfer with Parameter Control

Now let's create an interactive widget to experiment with different parameters:

In [ ]:
# Create interactive widgets for parameter control
style_weight_slider = widgets.FloatLogSlider(
    value=1000000,
    base=10,
    min=3,  # 10^3 = 1000
    max=7,  # 10^7 = 10000000
    step=0.1,
    description='Style Weight:',
    style={'description_width': 'initial'}
)

content_weight_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Content Weight:',
    style={'description_width': 'initial'}
)

steps_slider = widgets.IntSlider(
    value=200,  # Reduced for demo purposes
    min=50,
    max=500,
    step=50,
    description='Steps:',
    style={'description_width': 'initial'}
)

learning_rate_slider = widgets.FloatSlider(
    value=0.01,
    min=0.001,
    max=0.1,
    step=0.001,
    description='Learning Rate:',
    style={'description_width': 'initial'}
)

preset_dropdown = widgets.Dropdown(
    options=['custom'] + list(PRESETS.keys()),
    value='balanced',
    description='Preset:',
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description='🎨 Run Style Transfer',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

output_area = widgets.Output()

def update_sliders_from_preset(change):
    """Update sliders when preset is changed."""
    preset_name = change['new']
    if preset_name in PRESETS:
        preset = PRESETS[preset_name]
        content_weight_slider.value = preset['content_weight']
        style_weight_slider.value = preset['style_weight']
        steps_slider.value = min(preset['steps'], 500)  # Cap at 500 for demo

preset_dropdown.observe(update_sliders_from_preset, names='value')

def run_style_transfer(button):
    """Run style transfer with current parameters."""
    with output_area:
        clear_output(wait=True)
        
        print("🎨 Starting Neural Style Transfer...")
        print(f"Parameters: Content={content_weight_slider.value}, Style={style_weight_slider.value}")
        print(f"Steps: {steps_slider.value}, Learning Rate: {learning_rate_slider.value}")
        
        # Update model weights
        nst.set_loss_weights(
            content_weight=content_weight_slider.value,
            style_weight=style_weight_slider.value
        )
        
        try:
            # Run style transfer
            stylized_tensor = nst.transfer_style(
                content_image=content_tensor,
                style_image=style_tensor,
                num_steps=steps_slider.value,
                learning_rate=learning_rate_slider.value,
                show_progress=True
            )
            
            # Display results
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            axes[0].imshow(tensor_to_pil(content_tensor))
            axes[0].set_title('Content Image', fontsize=12, fontweight='bold')
            axes[0].axis('off')
            
            axes[1].imshow(tensor_to_pil(style_tensor))
            axes[1].set_title('Style Image', fontsize=12, fontweight='bold')
            axes[1].axis('off')
            
            axes[2].imshow(tensor_to_pil(stylized_tensor))
            axes[2].set_title('Stylized Result', fontsize=12, fontweight='bold')
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            print("✅ Style transfer completed successfully!")
            
        except Exception as e:
            print(f"❌ Error during style transfer: {e}")

run_button.on_click(run_style_transfer)

# Display the interactive interface
print("🎛️ Interactive Neural Style Transfer Control Panel")
display(
    widgets.VBox([
        widgets.HTML("<h3>🎨 Neural Style Transfer Parameters</h3>"),
        preset_dropdown,
        content_weight_slider,
        style_weight_slider,
        steps_slider,
        learning_rate_slider,
        run_button,
        output_area
    ])
)

## 8. Preset Comparison

Let's compare different preset configurations to understand their effects:

In [ ]:
def compare_presets(preset_list=['quick', 'balanced', 'more_style'], steps_override=100):
    """Compare different presets side by side."""
    
    results = {}
    
    print(f"🔄 Comparing {len(preset_list)} presets...")
    
    for preset_name in preset_list:
        if preset_name not in PRESETS:
            print(f"Preset '{preset_name}' not found, skipping...")
            continue
            
        preset = PRESETS[preset_name]
        print(f"\n🎨 Running preset: {preset_name}")
        print(f"   {preset['description']}")
        print(f"   Content Weight: {preset['content_weight']}, Style Weight: {preset['style_weight']}")
        
        # Update model weights
        nst.set_loss_weights(
            content_weight=preset['content_weight'],
            style_weight=preset['style_weight']
        )
        
        try:
            # Run style transfer with reduced steps for comparison
            stylized = nst.transfer_style(
                content_image=content_tensor,
                style_image=style_tensor,
                num_steps=steps_override,
                learning_rate=0.01,
                show_progress=False
            )
            
            results[preset_name] = stylized
            print(f"   ✅ Completed!")
            
        except Exception as e:
            print(f"   ❌ Failed: {e}")
    
    # Display comparison
    if results:
        n_results = len(results)
        fig, axes = plt.subplots(2, max(3, n_results), figsize=(4*max(3, n_results), 8))
        
        # Top row: Original images
        axes[0, 0].imshow(tensor_to_pil(content_tensor))
        axes[0, 0].set_title('Content Image', fontsize=12, fontweight='bold')
        axes[0, 0].axis('off')
        
        axes[0, 1].imshow(tensor_to_pil(style_tensor))
        axes[0, 1].set_title('Style Image', fontsize=12, fontweight='bold')
        axes[0, 1].axis('off')
        
        # Hide extra axes in top row
        for i in range(2, max(3, n_results)):
            axes[0, i].axis('off')
        
        # Bottom row: Results
        for i, (preset_name, result) in enumerate(results.items()):
            axes[1, i].imshow(tensor_to_pil(result))
            axes[1, i].set_title(f'{preset_name.title()}\n{PRESETS[preset_name]["description"]}', 
                               fontsize=10, fontweight='bold')
            axes[1, i].axis('off')
        
        # Hide extra axes in bottom row
        for i in range(len(results), max(3, n_results)):
            axes[1, i].axis('off')
        
        plt.suptitle('Preset Comparison Results', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print("\n📊 Comparison Analysis:")
        print("• 'quick': Fast processing, basic quality")
        print("• 'balanced': Good balance of content and style")
        print("• 'more_style': Emphasizes artistic style over content preservation")
    else:
        print("❌ No successful results to display")

# Run the comparison
compare_presets()

## 9. Understanding Loss Functions

Let's dive deeper into how the loss functions work and track them during optimization:

In [ ]:
def detailed_style_transfer_with_loss_tracking(content_img, style_img, num_steps=100):
    """Run style transfer while tracking loss components."""
    
    # Reset model to balanced settings
    nst.set_loss_weights(content_weight=1.0, style_weight=1000000.0)
    
    # Extract features
    with torch.no_grad():
        content_features = nst.feature_extractor(nst.normalize_vgg(content_img))
        style_features = nst.feature_extractor(nst.normalize_vgg(style_img))
    
    # Create loss modules
    content_losses, style_losses = nst.create_loss_modules(content_features, style_features)
    
    # Initialize generated image
    generated_image = content_img.clone()
    generated_image.requires_grad_(True)
    
    # Optimizer
    optimizer = torch.optim.LBFGS([generated_image], lr=0.01)
    
    # Track losses
    total_losses = []
    content_losses_track = []
    style_losses_track = []
    step_count = [0]
    
    def closure():
        step_count[0] += 1
        generated_image.data.clamp_(0, 1)
        optimizer.zero_grad()
        
        # Extract features from generated image
        generated_features = nst.feature_extractor(nst.normalize_vgg(generated_image))
        
        # Compute losses
        total_loss, content_loss, style_loss = nst.compute_total_loss(
            generated_features, content_losses, style_losses
        )
        
        # Track losses
        if step_count[0] % 10 == 0:  # Track every 10 steps
            total_losses.append(total_loss.item())
            content_losses_track.append(content_loss)
            style_losses_track.append(style_loss)
        
        total_loss.backward()
        return total_loss
    
    print(f"🏃 Running {num_steps} optimization steps with loss tracking...")
    
    # Run optimization
    progress_bar = tqdm(range(num_steps), desc="Optimizing")
    for i in progress_bar:
        optimizer.step(closure)
        
        if len(total_losses) > 0:
            progress_bar.set_postfix({
                'Total Loss': f'{total_losses[-1]:.2e}',
                'Content': f'{content_losses_track[-1]:.2e}',
                'Style': f'{style_losses_track[-1]:.2e}'
            })
    
    with torch.no_grad():
        generated_image.clamp_(0, 1)
    
    return generated_image.detach(), total_losses, content_losses_track, style_losses_track

# Run detailed style transfer
result, total_losses, content_losses, style_losses = detailed_style_transfer_with_loss_tracking(
    content_tensor, style_tensor, num_steps=150
)

# Plot loss curves
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Loss curves
steps_axis = np.arange(0, len(total_losses) * 10, 10)

ax1.plot(steps_axis, total_losses, 'b-', linewidth=2, label='Total Loss')
ax1.set_title('Total Loss Over Time', fontsize=14, fontweight='bold')
ax1.set_xlabel('Optimization Steps')
ax1.set_ylabel('Loss Value')
ax1.grid(True, alpha=0.3)
ax1.legend()

ax2.plot(steps_axis, content_losses, 'g-', linewidth=2, label='Content Loss')
ax2.plot(steps_axis, style_losses, 'r-', linewidth=2, label='Style Loss')
ax2.set_title('Content vs Style Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Optimization Steps')
ax2.set_ylabel('Loss Value')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Show result
ax3.imshow(tensor_to_pil(content_tensor))
ax3.set_title('Original Content', fontsize=12, fontweight='bold')
ax3.axis('off')

ax4.imshow(tensor_to_pil(result))
ax4.set_title('Stylized Result', fontsize=12, fontweight='bold')
ax4.axis('off')

plt.tight_layout()
plt.show()

print("\n📈 Loss Analysis:")
print(f"• Initial Total Loss: {total_losses[0]:.2e}")
print(f"• Final Total Loss: {total_losses[-1]:.2e}")
print(f"• Loss Reduction: {(total_losses[0] - total_losses[-1]) / total_losses[0] * 100:.1f}%")
print(f"• Final Content Loss: {content_losses[-1]:.2e}")
print(f"• Final Style Loss: {style_losses[-1]:.2e}")

## 10. Advanced Experiments

### Experiment 1: Effect of Different Style Weights

In [ ]:
def style_weight_experiment():
    """Experiment with different style weights to show their effects."""
    
    style_weights = [10000, 100000, 1000000, 10000000]
    results = {}
    
    print("🧪 Experimenting with different style weights...")
    
    for weight in style_weights:
        print(f"\n🎨 Testing style weight: {weight:,}")
        
        nst.set_loss_weights(content_weight=1.0, style_weight=weight)
        
        try:
            result = nst.transfer_style(
                content_image=content_tensor,
                style_image=style_tensor,
                num_steps=100,  # Reduced for experiment
                learning_rate=0.01,
                show_progress=False
            )
            results[weight] = result
            print(f"   ✅ Complete")
        except Exception as e:
            print(f"   ❌ Failed: {e}")
    
    # Display results
    if results:
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        
        for i, (weight, result) in enumerate(results.items()):
            axes[i].imshow(tensor_to_pil(result))
            axes[i].set_title(f'Style Weight: {weight:,}', fontsize=12, fontweight='bold')
            axes[i].axis('off')
        
        plt.suptitle('Effect of Different Style Weights', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print("\n📊 Style Weight Analysis:")
        print("• Lower weights (10K): More content preservation, less style")
        print("• Medium weights (100K-1M): Balanced style transfer")
        print("• Higher weights (10M): Strong style application, content may be lost")

style_weight_experiment()

### Experiment 2: Progressive Style Transfer

In [ ]:
def progressive_style_transfer():
    """Show the progression of style transfer over time."""
    
    print("🎬 Creating progressive style transfer visualization...")
    
    # Reset to balanced settings
    nst.set_loss_weights(content_weight=1.0, style_weight=1000000.0)
    
    # Extract features
    with torch.no_grad():
        content_features = nst.feature_extractor(nst.normalize_vgg(content_tensor))
        style_features = nst.feature_extractor(nst.normalize_vgg(style_tensor))
    
    # Create loss modules
    content_losses, style_losses = nst.create_loss_modules(content_features, style_features)
    
    # Initialize generated image
    generated_image = content_tensor.clone()
    generated_image.requires_grad_(True)
    
    # Optimizer
    optimizer = torch.optim.LBFGS([generated_image], lr=0.01)
    
    # Capture intermediate results
    snapshots = []
    snapshot_steps = [0, 25, 50, 100, 150, 200]
    step_count = [0]
    
    def closure():
        step_count[0] += 1
        generated_image.data.clamp_(0, 1)
        optimizer.zero_grad()
        
        # Capture snapshots
        if step_count[0] in snapshot_steps:
            with torch.no_grad():
                snapshots.append((step_count[0], generated_image.clone()))
        
        # Extract features from generated image
        generated_features = nst.feature_extractor(nst.normalize_vgg(generated_image))
        
        # Compute losses
        total_loss, _, _ = nst.compute_total_loss(
            generated_features, content_losses, style_losses
        )
        
        total_loss.backward()
        return total_loss
    
    # Initial snapshot
    snapshots.append((0, generated_image.clone()))
    
    # Run optimization
    for i in tqdm(range(200), desc="Progressive Transfer"):
        optimizer.step(closure)
    
    with torch.no_grad():
        generated_image.clamp_(0, 1)
    
    # Display progression
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, (step, snapshot) in enumerate(snapshots[:6]):
        axes[i].imshow(tensor_to_pil(snapshot))
        axes[i].set_title(f'Step {step}', fontsize=14, fontweight='bold')
        axes[i].axis('off')
    
    plt.suptitle('Progressive Style Transfer Evolution', fontsize=18, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n🎬 Progressive Analysis:")
    print("• Step 0: Original content image")
    print("• Steps 25-50: Initial style patterns appear")
    print("• Steps 100-150: Style becomes more pronounced")
    print("• Step 200: Final stylized result with refined details")

progressive_style_transfer()

## 11. Performance Analysis and Tips

Let's analyze the performance characteristics and provide optimization tips:

In [ ]:
import time

def performance_analysis():
    """Analyze performance characteristics of different configurations."""
    
    print("⚡ Performance Analysis\n")
    
    # Test different image sizes
    image_sizes = [256, 512, 768]
    size_times = []
    
    print("📏 Testing different image sizes:")
    for size in image_sizes:
        print(f"\n  Testing {size}x{size} image...")
        
        # Resize test images
        test_content = load_and_preprocess_image(content_display_path, target_size=size, device=device)
        test_style = load_and_preprocess_image(style_display_path, target_size=size, device=device)
        
        nst.set_loss_weights(content_weight=1.0, style_weight=1000000.0)
        
        start_time = time.time()
        try:
            result = nst.transfer_style(
                content_image=test_content,
                style_image=test_style,
                num_steps=50,  # Reduced for timing
                learning_rate=0.01,
                show_progress=False
            )
            elapsed_time = time.time() - start_time
            size_times.append(elapsed_time)
            print(f"    ✅ Time: {elapsed_time:.2f} seconds")
            
            # Memory usage
            if torch.cuda.is_available():
                memory_used = torch.cuda.memory_allocated() / 1e9
                print(f"    💾 GPU Memory: {memory_used:.2f} GB")
            
        except Exception as e:
            print(f"    ❌ Failed: {e}")
            size_times.append(None)
    
    # Test different step counts
    step_counts = [50, 100, 200, 500]
    step_times = []
    
    print("\n🏃 Testing different step counts (512x512 image):")
    for steps in step_counts:
        print(f"\n  Testing {steps} steps...")
        
        start_time = time.time()
        try:
            result = nst.transfer_style(
                content_image=content_tensor,
                style_image=style_tensor,
                num_steps=steps,
                learning_rate=0.01,
                show_progress=False
            )
            elapsed_time = time.time() - start_time
            step_times.append(elapsed_time)
            print(f"    ✅ Time: {elapsed_time:.2f} seconds")
            print(f"    📊 Time per step: {elapsed_time/steps:.3f} seconds")
            
        except Exception as e:
            print(f"    ❌ Failed: {e}")
            step_times.append(None)
    
    # Visualize results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Image size vs time
    valid_size_data = [(size, time) for size, time in zip(image_sizes, size_times) if time is not None]
    if valid_size_data:
        sizes, times = zip(*valid_size_data)
        ax1.plot(sizes, times, 'bo-', linewidth=2, markersize=8)
        ax1.set_xlabel('Image Size (pixels)')
        ax1.set_ylabel('Time (seconds)')
        ax1.set_title('Processing Time vs Image Size\n(50 steps)', fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3)
    
    # Steps vs time
    valid_step_data = [(steps, time) for steps, time in zip(step_counts, step_times) if time is not None]
    if valid_step_data:
        steps, times = zip(*valid_step_data)
        ax2.plot(steps, times, 'ro-', linewidth=2, markersize=8)
        ax2.set_xlabel('Number of Steps')
        ax2.set_ylabel('Time (seconds)')
        ax2.set_title('Processing Time vs Steps\n(512x512 image)', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Performance Tips:")
    print("• GPU acceleration provides 10-50x speedup over CPU")
    print("• Processing time scales roughly quadratically with image size")
    print("• Processing time scales linearly with number of steps")
    print("• For experimentation: use 256-512px images with 100-200 steps")
    print("• For high quality: use 768-1024px images with 500-1000 steps")
    print("• Memory usage scales with image size squared")
    print("• LBFGS optimizer is more memory-efficient than Adam for this task")

performance_analysis()

## 12. Summary and Next Steps

Congratulations! You've completed a comprehensive exploration of Neural Style Transfer. Let's summarize what you've learned:

In [ ]:
# Create a summary visualization
def create_summary():
    """Create a summary of the Neural Style Transfer process."""
    
    print("📚 Neural Style Transfer - Complete Tutorial Summary")
    print("=" * 60)
    
    summary_points = [
        "🧠 Theoretical Understanding",
        "   • Neural Style Transfer combines content and style representations",
        "   • VGG19 extracts hierarchical features from images",
        "   • Content captured by direct feature comparison (conv4_2)",
        "   • Style captured by Gram matrices of feature correlations",
        "",
        "🔧 Implementation Details",
        "   • PyTorch-based implementation with pre-trained VGG19",
        "   • LBFGS optimizer for image optimization",
        "   • Combined loss function: α*L_content + β*L_style",
        "   • Iterative optimization from content image",
        "",
        "⚙️ Parameter Effects",
        "   • Content weight (α): Higher values preserve more content",
        "   • Style weight (β): Higher values apply more style",
        "   • Steps: More steps = better quality, longer processing",
        "   • Image size: Larger = higher quality, more memory/time",
        "",
        "🚀 Performance Optimization",
        "   • GPU acceleration essential for practical use",
        "   • Balance image size vs processing time",
        "   • Use presets for common scenarios",
        "   • Monitor memory usage for large images",
        "",
        "🎯 Next Steps",
        "   • Experiment with different style images",
        "   • Try real-world content images",
        "   • Explore fast style transfer methods",
        "   • Implement video style transfer",
        "   • Study perceptual loss functions"
    ]
    
    for point in summary_points:
        print(point)
    
    print("\n" + "=" * 60)
    print("🎉 Thank you for completing this Neural Style Transfer tutorial!")
    print("🌐 Explore more at: github.com/your-repo/neural-style-transfer")

create_summary()

# Display final example with best settings
print("\n🎨 Final Demonstration - High Quality Result:")

# Reset to high quality settings
nst.set_loss_weights(content_weight=1.0, style_weight=1000000.0)

final_result = nst.transfer_style(
    content_image=content_tensor,
    style_image=style_tensor,
    num_steps=300,
    learning_rate=0.01,
    show_progress=True
)

# Create final comparison
visualize_comparison(
    content_tensor, 
    style_tensor, 
    final_result,
    figsize=(18, 6)
)

# Save the final result
save_image(final_result, "outputs/notebook_final_result.jpg")
print("💾 Final result saved to: outputs/notebook_final_result.jpg")

## 📖 Additional Resources

### Papers and Research
- **Original Paper**: Gatys et al. "A Neural Algorithm of Artistic Style" (2015)
- **Fast Style Transfer**: Johnson et al. "Perceptual Losses for Real-Time Style Transfer" (2016)
- **Improved Quality**: Li et al. "Demystifying Neural Style Transfer" (2017)

### Extensions and Improvements
- **AdaIN**: Huang & Belongie "Arbitrary Style Transfer in Real-time with Adaptive Instance Normalization" (2017)
- **WCT**: Li et al. "Universal Style Transfer via Feature Transforms" (2017)
- **Avatar-Net**: Sheng et al. "Avatar-Net: Multi-scale Zero-shot Style Transfer" (2018)

### Practical Applications
- Mobile apps (Prisma, DeepArt)
- Video style transfer
- Interactive art creation
- Augmented reality filters
- Digital art generation

### Try These Experiments
1. **Different Style Images**: Try famous paintings, textures, or abstract art
2. **Video Style Transfer**: Apply to video frames for animated effects
3. **Multiple Styles**: Blend multiple style images
4. **Semantic Style Transfer**: Apply different styles to different regions
5. **3D Style Transfer**: Extend to 3D scenes and objects

---

*This notebook provided a comprehensive introduction to Neural Style Transfer. The techniques learned here form the foundation for many modern AI art applications. Happy experimenting!* 🎨✨